Here the M=1 single iteration for test isn't done and directly the M=50 iteration is performed

Reason : We've already gotten acquainted with the base method by performing Passive Learning Previously

Active Learning (M=50) (M=1 Test not done)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings("ignore")

X_y_train = pd.read_csv('Bank_Train.csv',index_col=0)
X_y_test = pd.read_csv('Bank_Test.csv',index_col=0)
X_test = X_y_test.drop(columns=['y'])
y_test = X_y_test['y']
C_range = np.array([10**(-3),10**(-2),10**(-1),10**(0),10**(1),10**(2),10**(3),10**(4),10**(5),10**(6)])
Error_Test_df = pd.DataFrame(index=np.arange(1,51),columns=np.arange(10,910,10))

In [2]:
for mc in np.arange(1,51):    
    
    print(mc)     # Status Check
    
    X_y_train_updated = X_y_train.copy()

    X_y_train_build = pd.DataFrame()                     




    for i in np.arange(1,91):     
        
        if i == 1:
            
            X_y_train_to_add, X_y_train_updated = train_test_split(X_y_train_updated,train_size=10,shuffle=True,stratify=X_y_train_updated['y'])
            
            X_y_train_to_add.reset_index(drop=True,inplace=True)
            X_y_train_updated.reset_index(drop=True,inplace=True)

            
        elif i != 1 and i != 90:
            
            
            samp_indic = np.arange(0,X_y_train_updated.shape[0])
            
            indx_sel_df = pd.DataFrame(np.transpose(np.array([samp_indic,hypr_dist])),columns=['index','dist'])
            
            sort_indx_sel_df = indx_sel_df.sort_values(by=['dist'],ascending=True)
            
            sel_indx_arry = sort_indx_sel_df.iloc[0:10,0].to_numpy().flatten()
            
            indx_rmn_arry = sort_indx_sel_df.iloc[10:,0].to_numpy().flatten()
            
            X_y_train_to_add = X_y_train_updated.iloc[sel_indx_arry,:]
            
            X_y_train_updated = X_y_train_updated.iloc[indx_rmn_arry,:]
            
            
            X_y_train_to_add.reset_index(drop=True,inplace=True)
            X_y_train_updated.reset_index(drop=True,inplace=True)

            
        else:
            
            X_y_train_to_add = X_y_train_updated



        X_y_train_build = pd.concat([X_y_train_build,X_y_train_to_add],axis=0)

        X_y_train_build.reset_index(drop=True,inplace=True)



        CV_errors_df = pd.DataFrame(index=np.arange(0,5),columns=C_range) 

        cv_ite = -1

        X_train_build = X_y_train_build.drop(columns=['y'])
        y_train_build = X_y_train_build['y']


        kf = StratifiedKFold(n_splits=5,shuffle=True)       


        for train,test in kf.split(X_train_build,y_train_build):

            cv_ite = cv_ite + 1

            X_train_cv, X_test_cv = X_train_build.iloc[train,:],X_train_build.iloc[test,:]
            y_train_cv, y_test_cv = y_train_build.iloc[train],y_train_build.iloc[test]


            for j in np.arange(0,C_range.size):

                svm_cv = LinearSVC(penalty='l1',dual=False,C=C_range[j]).fit(X_train_cv,y_train_cv)

                y_test_cv_predicted = svm_cv.predict(X_test_cv)

                mis_loc = 0

                for l in np.arange(0,X_test_cv.shape[0]):

                    if y_test_cv_predicted[l] != y_test_cv.iloc[l]:

                        mis_loc = mis_loc + 1

                err_loc = (mis_loc/X_test_cv.shape[0])*100

                CV_errors_df.iloc[cv_ite,j] = err_loc



        CV_mean_errors = CV_errors_df.mean(axis=0)

        error_vector = CV_mean_errors.to_numpy().flatten()

        indx_C_selected = np.argmin(error_vector)

        min_C = C_range[indx_C_selected]



        svm_f = LinearSVC(penalty='l1',dual=False,C=min_C).fit(X_train_build,y_train_build)

        y_test_predicted = svm_f.predict(X_test)
        
        hypr_dec_func = svm_f.decision_function(X_y_train_updated.drop(columns=['y']))
        
        hypr_dist = abs(hypr_dec_func)
        
        mis = 0

        for k in np.arange(0,X_test.shape[0]):

            if y_test_predicted[k] != y_test.iloc[k]:

                mis = mis + 1

        err = (mis/X_test.shape[0])*100


        Error_Test_df.iloc[mc-1,i-1] = err





print(Error_Test_df)    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
        10       20        30        40        50        60        70   \
1   2.33051  3.60169   1.48305   1.05932   1.48305   1.05932   1.05932   
2   2.75424  7.62712    2.9661   1.27119   1.05932   1.05932   1.05932   
3   2.11864   6.5678   2.33051  0.423729   4.66102  0.211864   1.05932   
4   13.9831  2.75424   2.33051   2.11864   1.48305   1.05932   1.05932   
5   18.0085  6.35593   4.87288   2.33051   1.05932   1.05932   1.05932   
6   24.3644  1.90678   1.48305   1.05932   1.05932   1.05932   1.05932   
7   1.05932  1.05932   1.05932   1.05932   1.05932   1.05932   1.05932   
8   4.44915  10.3814  0.635593  0.423729  0.211864   1.05932   1.05932   
9    21.822  3.81356   2.54237   1.69492   1.27119   1.05932   1.05932   
10  5.08475  10.3814    6.5678   2.54237   1.90678  0.635593  0.423729   
11  8.26271  1.48305   1.69492   1.05932   1.

In [3]:
Error_Test_df.to_csv('Active_Learning_Errors.csv')